# Informe de Data Understanding

## Descripción de Datos

In [2]:
import pandas as pd

categoria = pd.read_csv("categoria.csv")
cliente = pd.read_csv("cliente.csv", encoding="latin1")
eventos = pd.read_csv("events.csv")
marca = pd.read_csv("marca.csv")
producto = pd.read_csv("producto.csv")

def resumen_tabla(nombre, df):
    print(f"\n--- {nombre.upper()} ---")
    print("Columnas y tipos de datos:")
    print(df.dtypes)
    print(df.head())

# Mostrar resumen de cada tabla
resumen_tabla("categoria", categoria)
resumen_tabla("cliente", cliente)
resumen_tabla("eventos", producto)
resumen_tabla("marca", marca)
resumen_tabla("producto", producto)




--- CATEGORIA ---
Columnas y tipos de datos:
id            int64
categoria    object
dtype: object
   id                       categoria
0   1                 SCOTCH WHISKIES
1   2       STRAIGHT BOURBON WHISKIES
2   3                BLENDED WHISKIES
3   4               IMPORTED DRY GINS
4   5  DECANTERS & SPECIALTY PACKAGES

--- CLIENTE ---
Columnas y tipos de datos:
id            float64
nombre         object
apellido       object
nacimiento     object
genero         object
empresa        object
idioma         object
nit            object
puesto         object
ciudad         object
correo         object
telefono       object
dtype: object
         id   nombre  apellido nacimiento  genero       empresa   idioma  \
0  599528.0   Samuel      Ward     4/6/89    Male        Yakijo  Marathi   
1  121688.0   Willie  Gonzales    6/29/72    Male     Zoonoodle  Maltese   
2  552148.0    Betty   Spencer     9/2/83  Female       Youtags  Dhivehi   
3  102019.0  Beverly    Jordan    1/15/72  Fem

Cada tabla cuenta con diferentes datos. Inicialmente, Categoria solamente cuenta con ID y con la categoria a la que pertenece dicho ID. Luego, viene cliente, una tabla extensa. Esta cuenta con información del cliente como su nombre, correo, genero, idioma, residencia, etc. Esta tabla ayuda a tener esta información, que puede llegar a ser sensible, de los clientes.

Luego, eventos, que indica un tracker de un sistema de compras. Este tracker contiene la información como hora, quién hizo la acción, que acción hizo, el item que estuvo en la acción y , si hubo transacción, el ID de esta. Marca, se parece a la tabla de Cuenta que solo tiene su ID y la marca a la que pertenece. Por ultimo, Producto que contiene la categoria, la marca el nombre, el volumen y el precio del item. 

El tipo de cada dato se puede visualizar en el chunk anterior, donde se muestran tambien 5 registros por tabla. 



## Estado en la Calidad de Datos

In [3]:
def calidad(nombre, df):
    print(f"\n--- {nombre.upper()} ---")
    print("Valores nulos por columna:")
    print(df.isnull().sum())
    print(f"\nFilas duplicadas: {df.duplicated().sum()}")

# Evaluar cada tabla
calidad("categoria", categoria)
calidad("cliente", cliente)
calidad("eventos", eventos)
calidad("marca", marca)
calidad("producto", producto)


--- CATEGORIA ---
Valores nulos por columna:
id           0
categoria    0
dtype: int64

Filas duplicadas: 0

--- CLIENTE ---
Valores nulos por columna:
id            281
nombre        281
apellido      281
nacimiento    281
genero        281
empresa       281
idioma        281
nit           281
puesto        281
ciudad        281
correo        281
telefono      281
dtype: int64

Filas duplicadas: 280

--- EVENTOS ---
Valores nulos por columna:
timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64

Filas duplicadas: 460

--- MARCA ---
Valores nulos por columna:
id       0
marca    0
dtype: int64

Filas duplicadas: 0

--- PRODUCTO ---
Valores nulos por columna:
id                 0
categoria_id    1028
nombre             0
marca_id         876
volumen            0
precio             6
dtype: int64

Filas duplicadas: 0


En cuanto a la calidad de datos de las tablas, estas difieren. En el caso de categoria, no tiene ni valores faltantes ni repetidos. En cuanto a cliente, existen 281 nulos y 280 repetidos. Esto se debe a que existen 281 registros completamente vacios. Al seguir con el laboratorio adelante solamente se tomarán los registros que sí tienen datos.

En cuanto a eventos, se puede ver que no existen valores nulos en si, pues transactionid solo existe si la compra se realiza. Esto quiere decir que los 460 registros duplicados, puede deberse a varios motivos. Estos motivos pueden ser que la captura de eventos es errónea, por ejemplo, si una persona no hace todo el proceso en un solo momento, sino que toma un tiempo entre sus decisiones. Esto puede crear problemas al duplicar los registros.

Con marca, no hay registros ni repetidos ni nulos. Con producto, existen varios que no tienen ni producto no marca, aparte de los 6 que no tienen precio. Esto se da debido a que varios productos son llamados "genéricos" donde no tienen estos campos en sus registros.

Para seguir adelante, se quitaran los registros completamente vacios, se quitaran los duplicados y en el caso de producto, se tomaran solo los que tienen precio, que es un campo mas importante que marca y categoria.

In [4]:
# Quitar registros vacios (clientes)
cliente = cliente.dropna(how="all")

# Quitar duplicados
eventos = eventos.drop_duplicates()

# Quitar registros sin precio
producto = producto.dropna(subset=["precio"])



## Distribuciones y estadísticas

In [5]:
# Función para obtener estadísticas descriptivas
def describir_tabla(nombre, df):
    print(f"\n=== {nombre.upper()} ===")
    print("Estadísticas")
    print(df.describe(include=[object]))

# Usar las tablas limpias
describir_tabla("categoria", categoria)
describir_tabla("producto", producto) 
describir_tabla("marca", marca)
describir_tabla("cliente", cliente)  
describir_tabla("events", eventos)  




=== CATEGORIA ===
Estadísticas
              categoria
count               101
unique              101
top     SCOTCH WHISKIES
freq                  1

=== PRODUCTO ===
Estadísticas
               nombre
count           12020
unique           8211
top     Generic Drink
freq              876

=== MARCA ===
Estadísticas
                  marca
count               307
unique              307
top     Diageo Americas
freq                  1

=== CLIENTE ===
Estadísticas
        nombre apellido nacimiento  genero empresa     idioma          nit  \
count    11719    11719      11719   11719   11719      11719        11719   
unique    4648     6467       7547       2     383         97        11719   
top     Angela   Morris    8/31/93  Female   Voomm  Mongolian  411-44-7088   
freq        42       33          7    5895      71        143            1   

                      puesto     ciudad           correo          telefono  
count                  11719      11719            11719     

## Observaciones iniciales

### Insights clave para futuros proyectos de Machine Learning
1. Los datos presentan una variedad de fuentes y estructuras, incluyendo tablas de referencia simples (categoría, marca) y otras más complejas y ricas en información (cliente, producto, eventos).

2. La información de clientes y productos es detallada, permitiendo segmentaciones y análisis profundos, aunque existen registros genéricos o incompletos que reflejan la diversidad y heterogeneidad de los datos originales.

3. Los eventos de usuario capturan múltiples interacciones en el sistema de compras, lo que genera un volumen considerable de registros y la posibilidad de analizar patrones de comportamiento y trayectorias de usuario.

4. La integración entre tablas se apoya en identificadores comunes, lo que facilita el cruce de información y la construcción de vistas unificadas para análisis avanzados.

5. El proceso de limpieza y transformación de los datos ha sido documentado, permitiendo reproducir el flujo de trabajo y entender las decisiones tomadas en cada etapa del procesamiento.

## Posibles Proyectos con Data
1. **Shopper Personas: Segmentación inteligente de clientes**  
A partir de los datos demográficos y de comportamiento de los clientes, este proyecto busca identificar grupos o segmentos con características y patrones de compra similares. El resultado permitirá personalizar campañas de marketing y mejorar la experiencia del usuario.

2. **CompraFutura: Predicción de intención de compra**  
Utilizando el historial de eventos y atributos de clientes y productos, este modelo estima la probabilidad de que un usuario realice una compra en el futuro cercano. Esto puede ayudar a anticipar la demanda y diseñar estrategias de retención.

3. **Recomiéndame Algo: Sistema de recomendación de productos**  
Basado en las interacciones previas de los usuarios y las características de los productos, este sistema sugiere artículos relevantes para cada cliente, incrementando la tasa de conversión y la satisfacción del usuario.

## Limitaciones

**Shopper Personas:**
- La presencia de registros incompletos o genéricos en la tabla de clientes puede dificultar la identificación precisa de segmentos y reducir la representatividad de algunos grupos.
- La falta de variables adicionales sobre comportamiento fuera de compras (por ejemplo, feedback a productos o general) limita la profundidad de los perfiles que se pueden construir.

**CompraFutura:**
- La existencia de duplicados y posibles inconsistencias en los eventos puede introducir ruido en el entrenamiento del modelo y afectar la precisión de las predicciones.
- La información temporal de los eventos puede no ser suficiente para capturar patrones de recurrencia o estacionalidad si los datos no cubren un periodo amplio o continuo.

**Recomiéndame Algo:**
- La escasez de interacciones para ciertos usuarios o productos (cold start) puede dificultar la generación de recomendaciones personalizadas efectivas.
- La falta de atributos detallados en algunos productos (por ejemplo, productos genéricos sin marca o categoría) puede limitar la capacidad del sistema para sugerir artículos relevantes basados en contenido.

## Informacion complementaria

- Datos de navegación o interacción en la web/app (páginas visitadas, tiempo en cada sección, búsquedas realizadas).
- Feedback o valoraciones de los clientes sobre productos o servicios.
- Información transaccional más detallada, como métodos de pago, devoluciones o cancelaciones.
- Variables contextuales, como campañas de marketing activas, descuentos aplicados o eventos externos que puedan influir en el comportamiento de compra.
- Datos temporales más extensos para analizar tendencias a largo plazo o estacionalidad.
- Información sociodemográfica adicional de los clientes (nivel socioeconómico, ocupación, etc.).
- Datos de inventario o disponibilidad de productos en el momento de la compra.